In [52]:
!pip install -U -q google-generativeai

In [53]:
# importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('MY_GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [54]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [55]:
# Exemplo de embedding
title = "Um exemplo de incorporação de texto"
sample_text = ("Título: Um exemplo de incorporação de texto"
"\n"
"Texto completo:\n"
"\n"
"Este é um exemplo de como incorporar texto usando uma API de IA generativa.")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.03598598, -0.06447236, -0.035438538, 0.030003397, 0.04562859, 0.013494779, 0.02626964, -0.0011260518, 0.023212044, 0.057821307, 0.027998667, 0.015357801, -0.011836419, -0.014513524, -0.0050389008, -0.012967192, -0.002879948, 0.030850863, -0.03590793, 0.02114287, -0.012625597, 0.014075397, -0.043177832, -0.048292924, 0.018615492, 0.011776468, 0.011079586, -0.041057095, -0.02682171, 0.0044239038, -0.03369675, 0.08763656, -0.013151731, 0.019348357, 0.0026415605, -0.024945725, -0.042829055, -0.0077682934, -0.016561383, -0.006603864, -0.010767825, -0.05925673, -0.007980078, 0.022391409, -0.009059564, -0.029844811, 0.030236514, 0.011698594, 0.011928986, -0.061625496, 0.0017352866, -0.007669209, 0.028256254, -0.01669407, -0.007341815, -0.023472846, 0.04868491, 0.001442024, -0.01669035, -0.011326488, -0.003928877, 0.016342366, 0.0061267125, 0.009491787, 0.020682782, -0.061092097, -0.03182078, 0.040118955, 0.03537573, 0.0016058242, -0.0011248398, -0.03763439, 0.068160534, 0.002

In [56]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "A Jornada de Luke Skywalker",
    "Conteúdo": "Em uma galáxia repleta de perigos e intrigas, Luke Skywalker emerge como um improvável herói. Criado em um planeta desértico, ele descobre sua conexão com a Força e se torna um jedi. Sua jornada o leva a enfrentar o Império Galáctico, confrontar seu próprio passado e lutar pela liberdade de toda a galáxia."}

DOCUMENT2 = {
    "Título": "O Conflito Intergaláctico",
    "Conteúdo": "A guerra entre o Império e a Aliança Rebelde atinge seu auge. Naves espaciais colidem, sabres de luz brilham e planetas são devastados. Os siths tramam em busca de poder, enquanto os jedis resistem. O destino da galáxia está em jogo, e cada escolha tem consequências profundas."}

DOCUMENT3 = {
    "Título": "Os Mistérios da Força",
    "Conteúdo": "A Força permeia tudo: vida, morte, luz e escuridão. Os jedis buscam compreender seus segredos, enquanto os siths a exploram para ganho pessoal. A dualidade da Força é um tema central, e personagens como Yoda e Darth Vader personificam essa luta interna. A verdadeira natureza da Força permanece um mistério, mesmo em uma galáxia tão vasta."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [57]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,A Jornada de Luke Skywalker,"Em uma galáxia repleta de perigos e intrigas, ..."
1,O Conflito Intergaláctico,A guerra entre o Império e a Aliança Rebelde a...
2,Os Mistérios da Força,"A Força permeia tudo: vida, morte, luz e escur..."


In [58]:
model="models/embedding-001"
task_type="RETRIEVAL_DOCUMENT"

In [59]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [60]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,A Jornada de Luke Skywalker,"Em uma galáxia repleta de perigos e intrigas, ...","[0.018315682, 0.0031394516, -0.009813575, 0.02..."
1,O Conflito Intergaláctico,A guerra entre o Império e a Aliança Rebelde a...,"[0.016300356, 0.0018369316, -0.027330514, 0.00..."
2,Os Mistérios da Força,"A Força permeia tudo: vida, morte, luz e escur...","[0.032309636, 0.012862907, 0.016274406, -0.017..."


In [63]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [64]:
consulta = "Uma coisa invisível e misteriosa que está em todos os lugares, que dá poder tanto bom quanto mau."

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A Força permeia tudo: vida, morte, luz e escuridão. Os jedis buscam compreender seus segredos, enquanto os siths a exploram para ganho pessoal. A dualidade da Força é um tema central, e personagens como Yoda e Darth Vader personificam essa luta interna. A verdadeira natureza da Força permanece um mistério, mesmo em uma galáxia tão vasta.


In [65]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [66]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

A Força é tipo o Wi-Fi do universo, tá em tudo: na vida, na morte, na luz e na escuridão. Os jedis são tipo os nerds que querem entender como ela funciona, enquanto os siths são os malvadões que usam ela pra fazer bagunça.

É tipo uma moeda com dois lados, essa Força. Tem o Yoda, que é o cara do bem, e o Darth Vader, que é o cara do mal. Eles são tipo o yin e o yang da Força, sempre brigando lá dentro.

E o mais legal é que ninguém sabe direito o que é a Força. É tipo um enigma cósmico, mesmo numa galáxia tão grande quanto a nossa.
